In [1]:
import pandas as pd
import os
import cv2
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
data = pd.read_csv('./data/metadata.csv', index_col=None)

In [3]:
fundus_subset = data[(data['fundus'].notnull()) & (data['fundus'] != -1) & (data['types'] != -1)]
fundus_subset = fundus_subset[['types', 'fundus', 'names']]

fundus_subset['combined'] = fundus_subset['names'] + '.png'

fundus_subset['fundus'] = fundus_subset['fundus'].astype(str)

fundus_subset['types'] = fundus_subset['types'].astype(int)

In [4]:
fundus_subset.head()

,types,fundus,names,combined
0,0,/full-fundus/OIA-ODIR-TEST-OFFLINE-1.png,OIA-ODIR-TEST-OFFLINE-1,OIA-ODIR-TEST-OFFLINE-1.png
1,0,/full-fundus/OIA-ODIR-TEST-OFFLINE-2.png,OIA-ODIR-TEST-OFFLINE-2,OIA-ODIR-TEST-OFFLINE-2.png
2,0,/full-fundus/OIA-ODIR-TEST-OFFLINE-4.png,OIA-ODIR-TEST-OFFLINE-4,OIA-ODIR-TEST-OFFLINE-4.png
3,0,/full-fundus/OIA-ODIR-TEST-OFFLINE-5.png,OIA-ODIR-TEST-OFFLINE-5,OIA-ODIR-TEST-OFFLINE-5.png
4,0,/full-fundus/OIA-ODIR-TEST-OFFLINE-6.png,OIA-ODIR-TEST-OFFLINE-6,OIA-ODIR-TEST-OFFLINE-6.png


In [5]:
# glaucoma fundus images
glaucomaCount = len(fundus_subset[(fundus_subset['types'] == 1)])
print(glaucomaCount)

# healthy fundus images
healthyCount = len(fundus_subset[(fundus_subset['types'] == 0)])
print(healthyCount)

# randomize data
fundus_subset_random = fundus_subset.sample(frac=1, random_state=1)

4767
7549


In [6]:
# create df for each class based on randomized data
healthy_subset = fundus_subset_random[fundus_subset_random['types'] == 0]
glaucoma_subset = fundus_subset_random[fundus_subset_random['types'] == 1]

In [7]:
testSize = 0.1 * len(glaucoma_subset)

In [8]:
glaucoma_subset[['fundus', 'types']].reset_index().drop(columns='index')

,fundus,types
0,/full-fundus/EyePACS-Glaucoma-2749.png,1
1,/full-fundus/G1020-967.png,1
2,/full-fundus/sjchoi86-HRF-310.png,1
3,/full-fundus/EyePACS-Glaucoma-898.png,1
4,/full-fundus/EyePACS-Glaucoma-182.png,1
...,...,...
4762,/full-fundus/EyePACS-Glaucoma-218.png,1
4763,/full-fundus/EyePACS-Glaucoma-1168.png,1
4764,/full-fundus/EyePACS-Glaucoma-2764.png,1
4765,/full-fundus/G1020-607.png,1


In [9]:
glaucoma_subset.iloc[0]['types']

1

In [10]:
import sys
sys.path.append('./models')
from dataset import GlaucomaLDM

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [11]:
datamodule = GlaucomaLDM(batch_size=2)

In [12]:
datamodule.setup("fit")

In [13]:
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
test_loader = datamodule.test_dataloader()

In [14]:
len(test_loader), len(val_loader)

(476, 429)

In [15]:
len(train_loader)

3862

In [16]:
image, label = next(iter(train_loader))

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


(3, 512, 512)
(512, 512, 3)
torch.Size([3, 512, 512])
(3, 512, 512)
(512, 512, 3)
torch.Size([3, 512, 512])


In [17]:
image.shape
image.dtype

torch.float32

In [18]:
from models import U_Net

In [19]:
model = U_Net()
model(image).shape

torch.Size([2, 1])

In [20]:
sys.path.append('./models/Optic-Disk-Cup-Segmentation/src/segmentation code')
from models import FCDenseModified
model2 = FCDenseModified(in_channels=3)
model2(image).shape
model2(image)

tensor([[0.5028],
        [0.5151]], grad_fn=<SigmoidBackward0>)

In [21]:
from models import EarlyId
model3 = EarlyId(3)

In [22]:
model3(image).shape

torch.Size([2, 1])

In [23]:
from models import LightningModel
lm = LightningModel("U-Net", None)

In [24]:
import torch
loss = torch.nn.BCELoss()

In [25]:
out = model(image)

In [26]:
out, label
label.resize_(out.size(dim=0), 1)
loss(out, label.to(torch.float))

tensor(0.8190, grad_fn=<BinaryCrossEntropyBackward0>)

In [32]:
sigmoid = torch.nn.Sigmoid()
sigmoid(out)

tensor([[0.6327],
        [0.6397]], grad_fn=<SigmoidBackward0>)

In [27]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
acc = BinaryAccuracy()

In [37]:
out, label
torch.any(label == 1)
#acc(out, label)

tensor(False)